In [1]:
import sympy
import numpy
from sympy import *
import math
#from sympy import Rational
#from sympy import poly

In [2]:

### This code contains the XK class
### XK objects contain the following attrubutes
## gen_0 is a list of two element lists. Each element is a generator. The two element lists are [gr_w,gr_z]
## gen_1 is a list of Maslov gradings.
##diff_0, diff_1, v_map and h_map are instances of the "mor" class, described below.


##The code has a lot of "magic methods" (operator override)
## so if X and Y are XK complexes then we can just type X*Y for their tensor product.


class XK(object):
    def __init__(self,gens_0,gens_1,diff_0,diff_1,v_map,h_map):
        self.gens_0=gens_0
        self.gens_1=gens_1
        self.diff_0=diff_0
        self.diff_1=diff_1
        self.v_map=v_map
        self.h_map=h_map
        self.rank_0=len(gens_0)
        self.rank_1=len(gens_1)
        self.id_0=endo(identity_map(self.rank_0))
        self.id_1=endo(identity_map(self.rank_1))
        self.Alex=[(x[0]-x[1])/2 for x in gens_0]
        
    
    ##The following is the index shift function, which is useful in the mapping cone complex below.
    ##This code is designed specifically for the mapping cone formula
    ## works with +1 and -1 framing
    def ind_shift(self,s,truncation,idempotent,framing):
        N=truncation
        rank_0=self.rank_0
        rank_1=self.rank_1
        ##we assume the complex is A_{-N},... A_N, B_{-N+framing},...B_N
        if idempotent==0:
            return (s+N)*rank_0
        if idempotent==1:
            return (2*N+1)*rank_0+(s+N-framing)*rank_1

        
    ##The following is the index shift function
    ##This code is designed specifically for the XImu_{+1}
    ## and won't work for a general framings
    ##We have made this part of the XKI class for usability
    ##This will only give the shift in indext for the 00 and 10 idempotents
    ##(i.e. the knot complex of the dual knot) for idempotents 
    
    def ind_shift_mu(self,s,truncation,idempotent,framing):
        N=truncation
        rank_0=self.rank_0
        rank_1=self.rank_1
        ##we assume the summands of the truncation dual knot mapping cone formula
        ## take the form
        ## A_{-N-1},... A_N, B_{-N-1+framing},...B_N
        if idempotent==0:
            return (s+N+1)*rank_0
        if idempotent==1:
            return (2*N+2)*rank_0+(s+N+1-framing)*rank_1
        
        
    
    #takes XK_complex and returns the underlying complex in idempotent 0
    def CFK(self):
        return CFK(self.gens_0,self.diff_0)

    def CFT(self):
        return CFT(self.gens_1,self.diff_1)
    
        
    def __mul__(self,X2):
        return XK_tensor(self,X2)   
    
 
    

                
        
        
##CF class is a class for free complexes over F[U] with Z-gradings
#gens should be a list of Maslov gradings (of the generators over F[U])
#diff is a list of sets, of the same length as gens.
#e.g. [set(),{1,2},set(),set(). in diff[i] we put all of the indices of the intersection points such that there is 
# a nonzero differential from i to j (the U-powers are filled in later).
#A variety of functions later on require a CF_complex to have
##just one F[U] tower. assume that the complex inputted has HF^infty equal to one tower.
## Also, diff is a list of sets. (List of list functionality has been removed)
        
class CF(object):
    def __init__(self,gens,diff):
        self.gens=gens
        self.diff=diff
        self.rank=len(gens)
    def dual(self):
        new_gens=[-x for x in self.gens]
        return CF(new_gens,self.diff.dual())
    ##This returns the dual complex. Note that this is a unitary command, so X-Y shouldn't work.
    def __neg__(self):
        return self.dual()
    ##Returns the direct sum of two complexes, indexed with the left
    ##complex starting at index 0 and the right with indices shifted
    def __add__(self,C2):
        gens1=self.gens
        gens2=C2.gens
        rank1=self.rank
        new_gens=gens1+gens2
        diff_matrix=[{i for i in x} for x in self.diff.matrix]
        diff_matrix.extend([{i+rank1 for i in x} for x in C2.diff.matrix])
        return CF(new_gens, endo(diff_matrix))
    ##This shifts the grading *up* by n
    ## This modifies the complex C
    def gr_shift(self, n):
        self.gens=[x+n for x in self.gens]
        return self
    
    def CF_old(self):
        new_gens=[x for x in self.gens]
        new_diff=[x for x in self.diff.matrix]
        return CF_old(new_gens, new_diff)

    
    
    
###CFT is a class for free F[U,T,T^{-1}] complexes which admit an Alexander grading
### These are essentially idempotent 1 complexes of XK modules for knots in integer homology spheres
### gens is a list of pairs [m,a] where m is the Maslov grading and a is the Alexander grading
### 
class CFT(object):
    def __init__(self,gens,diff):
        self.gens=gens
        self.diff=diff
        self.rank=len(gens)
    def dual(self):
        new_gens=[[-x[0],-x[1]] for x in self.gens]
        return CF(new_gens,self.diff.dual())
    ###The following gives the tensor product of two CFT_complexes
    ###and allows the notation C1*C2 for tensor products
    def __mul__(self,C2):
        return CFT_tensor(self,C2)
    def __neg__(self):
        return self.dual()
    ##returns underlying CF complex (i.e. just deletes the Alexander grading)
    def CF(self):
        new_gens=[g[0] for g in C.gens]
        return CF(new_gens, C.diff.copy())
    ##returns direct sum of self and C2
    def __add__(self,C2):
        gens1=self.gens
        gens2=C2.gens
        rank1=self.rank
        new_gens=gens1+gens2
        diff_matrix=[{i for i in x} for x in self.diff.matrix]
        diff_matrix.extend([{i+rank1 for i in x} for x in C2.diff.matrix])
        return CFT(new_gens, endo(diff_matrix))


###This class is the old (i.e. from an old code project) version of the CF class,
####where generators are lists (instead of mor's)
    
class CF_old(object):
    def __init__(self,gens,diff):
        self.gens=gens
        self.diff=diff
        self.rank=len(gens)
    def dual(self):
        new_gens=[-x for x in self.gens]
        dual_map_matrix=endo(self.diff).dual().matrix
        return CF(new_gens,dual_map_matrix)



    



##This is class CFK
###gens is a list of lists [gr_w,gr_z]
## diff is an object of class mor.
    
class CFK(object):
    def __init__(self,gens,diff):
        self.gens=gens
        self.diff=diff
        self.rank=len(gens) 
    def dual(self):
        new_gens=[[-x[0],-x[1]] for x in self.gens]
        return CFK(new_gens,self.diff.dual())
    ###The following gives the tensor product of two CFK_complexes
    ###and allows the notation C1*C2 for tensor products
    def __mul__(self,C2):
        return CFK_tensor(self,C2)
    def __neg__(self):
        return self.dual()
    #identical to CF_w below:
    def CF_w(self):
        new_gens=[g[0] for g in C.gens]
        return CF(new_gens, C.diff.copy())
    def CF_z(self):
        new_gens=[g[1] for g in C.gens]
        return CF(new_gens, C.diff.copy())
    ##returns direct sum of self and C2
    def __add__(self,C2):
        gens1=self.gens
        gens2=C2.gens
        rank1=self.rank
        new_gens=gens1+gens2
        diff_matrix=[{i for i in x} for x in self.diff.matrix]
        diff_matrix.extend([{i+rank1 for i in x} for x in C2.diff.matrix])
        return CFK(new_gens, endo(diff_matrix))
    def copy(self):
        gens=[[x for x in y] for y in self.gens]
        diff=self.diff.copy()
        return CFK(gens,diff)
    ## shift is a list of two elements
    def bi_gr_shift(self,shift):
        for x in self.gens:
            x[0]=x[0]+shift[0]
            x[1]=x[1]+shift[1]
    # here x is in the index of the generator, and we return its alexander grading
    def Alex(self,x):
        return (self.gens[x][0]-self.gens[x][1])//2
    
    # returns the complex As, as a CF_complex
    def As(self,s):
        new_gens=[]
        new_diff=self.diff.copy()
        for x in range(len(self.gens)):
            # we write either A(U^i x)=s or A(V^i x)=s depending on whether A(x)\ge s or A(x)\le s
            if self.Alex(x)<=s:
                ## then V^{s-A(x)} x is the generator
                new_gens.append(self.gens[x][0])
            if self.Alex(x)>s:
                ## then U^{A(x)-s} x is the gen
                new_gens.append(self.gens[x][0]-2*(self.Alex(x)-s))
        return CF(new_gens, new_diff)
    ## computes V_s(K)
    def Vs(self,s):
        As_old=self.As(s).CF_old()
        return -d_list(As_old)[0]//2
        
            
    
 
                
## returns tau, defined is the minimal Alexander grading of CFK(K)/W
    def tau(self):
        rank=self.rank
        gens=[x for x in self.gens]
        diff_C=self.diff.copy()
        ##  we now prune diff_C to include only the arrows which have W weight 0
        for x in range(rank):
            to_delete_x=set()
            for y in diff_C.matrix[x]:
                if self.gens[y][0]-self.gens[x][0]+1>0:
                    to_delete_x.add(y)
            diff_C.matrix[x]=diff_C.matrix[x].symmetric_difference(to_delete_x)
        diff=diff_C.matrix
        diff_dual=diff_C.dual().matrix
        arrow_length=0
        #first delete arrows of length 0, then 1 etc. each time we go through while loop
        ## we increase arrow_length by 1, until diff "is" zero.
        unpaired_generators_set={i for i in range(rank)}
        unpaired_generators_num=rank
        while unpaired_generators_num>1:
            to_remove=[]
            for x in unpaired_generators_set:
                known_short_arrow=false
                y_special=0
                for y in diff[x]:
                    if (gens[y][1]-gens[x][1]+1)/2==arrow_length:
                        known_short_arrow=true
                        y_special=y
                        break
                if known_short_arrow:
                    diff_x_non_special=[i for i in diff[x] if i!=y_special]
                    for y in diff_x_non_special:
                        basis_change_efficient(diff,diff_dual, y_special,y)
                    ###We are here going to make an edit and see if it improves efficiency
                    other_xs=[xo for xo in diff_dual[y_special]]
                    for x_other in other_xs:
                        if x_other!=x:
                            basis_change_efficient(diff,diff_dual, x_other,x)
                    to_remove.append(y_special)
                    to_remove.append(x)
                    unpaired_generators_num+=-2
            for x in to_remove:
                unpaired_generators_set.remove(x)
            arrow_length=arrow_length+1
    #        print(arrow_length)
        new_gens=[gens[i] for i in unpaired_generators_set]
        return (new_gens[0][0]-new_gens[0][1])//2



















        

            
##input is a list of pairs gens=[[x0,x1],...]
## and shift is a pair [shift0,shift1]
def bi_gr_shift(gens,shift):
    for x in gens:
        x[0]=x[0]+shift[0]
        x[1]=x[1]+shift[1]
    
##The following takes a CFKI complex and outputs the corresponding CFK/U and CFK/V complexes
## The outputs are 
def CF_w(C):
    new_gens=[g[0] for g in C.gens]
    return CF(new_gens, C.diff.copy())

def CF_z(C):
    new_gens=[g[1] for g in C.gens]
    return CF(new_gens, C.diff.copy())
    
    
    
##CFKI_to_XKI has input a CFKI complex 
## input is a CFKI complex
## assumed to be of a knot in S^3 (i.e. L-space type)
## output is the complex XK(S^3,K):
##We make idempotent 1 have a single generator. 
## Sets the complex to automatically have framing 0
## sets "model" to be 1.


def CFK_to_XK(C):
    gens_0=C.gens
    diff_0=C.diff
    gens_1=[[0,0]]
    diff_1=mor([{}],1)
    Cw=CF_w(C)
    Cz=CF_z(C)
    Rw=CF_reduced_SDR(Cw)
    Rz=CF_reduced_SDR(Cz)
    ##recall that the above are tuples
    ## [C_red,Pi,I,H]
    v_map=Rw[1]    
    h_map=Rz[1]
    return XK(gens_0,gens_1,diff_0,diff_1,v_map,h_map)
    
    



##Same as above but just with CFK complexes
def CFK_tensor(C1,C2):
    gens1=C1.gens
    gens2=C2.gens
    diff1=C1.diff
    diff2=C2.diff
    rank1=C1.rank
    rank2=C2.rank
    new_gens=[]
    for i in range(rank1):
        for j in range(rank2):
            new_gens.append([gens1[i][0]+gens2[j][0],gens1[i][1]+gens2[j][1]])
    new_diff=tensor(diff1,identity_mor(rank2))+tensor(identity_mor(rank1),diff2)
    return CFK(new_gens,new_diff)

##Same as above but just with CFT complexes (no actual difference in code)
def CFT_tensor(C1,C2):
    gens1=C1.gens
    gens2=C2.gens
    diff1=C1.diff
    diff2=C2.diff
    rank1=C1.rank
    rank2=C2.rank
    new_gens=[]
    for i in range(rank1):
        for j in range(rank2):
            new_gens.append([gens1[i][0]+gens2[j][0],gens1[i][1]+gens2[j][1]])
    new_diff=tensor(diff1,identity_mor(rank2))+tensor(identity_mor(rank1),diff2)
    return CFT(new_gens,new_diff)





#CFKI_tensor takes a list of objects of type CFI and returns their involutive tensor product
#assumes there is at least one element of the list
###Currently this is the fastest version.
def CFK_tensor_list(complex_list):
    new_complex=complex_list.pop(0)
    for remaining_complex in complex_list:
        new_complex=CFK_tensor(new_complex,remaining_complex)
    return new_complex



In [3]:
###This cell will contain the new "morphism" class,
##which is like the previous set based class but will allow easier implementation symbolically

##a morphism is just a pair [matrix,cod_rank]
## where matrix is a list of sets (the list based map, in previous notation) (i.e. the matrix of the morphism)
## and cod is the rank (number of generators) of the codomain




class mor(object):
    def __init__(self,matrix,co_rank=-1):
        self.matrix=matrix
        self.co_rank=co_rank
        if co_rank==-1:
            self.co_rank=len(matrix)
        self.rank=len(matrix)
    ### The purpose of this implementation is that we can write things like f+g and it will give the sum of the maps
    def __add__(self, mor2):
        rank=self.rank
        new_map=[self.matrix[i].symmetric_difference(mor2.matrix[i]) for i in range(rank)]
        return mor(new_map,self.co_rank)
    ## The purpose of this method is to return the composition of the two maps.
    def __mul__(self,mor2):
        new_map=[]
        G=self.matrix
        F=mor2.matrix
        for x in F:
            L=set()
            for Fx in x:
                L=L.symmetric_difference(G[Fx])
            new_map.append(L)
        return mor(new_map,self.co_rank)
    ##returns true if morphism is zero, and returns false ot
    def is_zero(self):
        return is_zero(self)
    def dual(self):
        return dual_mor(self)
    ## returns a copy of a matrix
    def copy(self):
        new_matrix=[{j for j in x} for x in self.matrix]
        return mor(new_matrix,self.co_rank)
    #the following method takes a morphism and adds an arrow from i to j 
    def add_arrow(self,i,j):
        self.matrix[i]=self.matrix[i].symmetric_difference({j})
    
    ## the function has input another mor, and integers start and shift
    ## we assume that f has smaller rank and co_rank
    ## (we do not check for valid output)
    ## what we do is view f as a block matrix, we start the index 0 of f at "start"
    ## and shift all the outputs by "shift"
    ## and then add the resulting matrix to self
    ## this modifies self. The idea is that this should be more efficient than
    ## making a sihift morphism and adding to f using +
    ## there is no return
    
    def add_block(self, f, start,shift):
        for i in range(f.rank):
            self.matrix[i+start]=self.matrix[i+start].symmetric_difference({j+shift for j in f.matrix[i]})

        
        
##The following has input a matrix, and returns the morphism which has co_rank=rank

def endo(matrix):
    return mor(matrix, len(matrix))

#is_zero checks whether a mor is [{},,,,{}]
def is_zero(M):
    L=M.matrix
    return L.count({})+L.count(set())==len(L)

## same as above except just a list of sets instead of a mor
def is_zero_old(L):
    return L.count({})+L.count(set())==len(L)


##The following takes two morphisms and returns their tensor product
##Note that this requires the knowledge of the co_rank

def tensor(M,N):
    F=M.matrix
    G=N.matrix
    new_map=[]
    for x1 in F:
        for x2 in G:
            T=set()
            for y1 in x1:
                for y2 in x2:
                    T.add(y1*N.co_rank+y2)
            new_map.append(T)
    return mor(new_map,M.co_rank*N.co_rank)


##For backwards compatibility
def sum_maps(f,g):
    return f+g
def compose_maps(f,g):
    return f*g

##function takes a list of morphisms and returns their sum
def sum_list(maps_list):
    new_map=maps_list[0]
    if len(maps_list)>0:
        for i in range(1,len(maps_list)):
            new_map=new_map+maps_list[i]
    return new_map


##Following code produces the dual map, if F is a map
## This assumes that F is a square matrix


##The following takes a mor Phi and returns the dual morphism
def dual_mor(Phi):
    F=Phi.matrix
    m=Phi.co_rank
    F_dual=[set() for i in range(m)]
    for i in range(len(F)):
        for j in F[i]:
            F_dual[j].add(i)
    return mor(F_dual,Phi.rank)


def identity_map(n):
    return [{i} for i in range(n)]

def identity_mor(n):
    return mor(identity_map(n),n)


##zero_mor gives the zero endomorphism
## m (optional) is the corange size. If not passed, then corange is assumed to be n.
##
def zero_mor(n,m=-1):
    if m==-1:
        return mor([set() for i in range(n)],n)
    else:
        return mor([set() for i in range(n)], m)
    
    
    
## We now have some elaborations of the above    
    
    
    
##################
###            ###
### Twisted cx ###
###            ###
##################
    
    
##Twisted_complex is a class. Obs is a collection of objects, and Mors is a mor_matrix
## Twisted_complexes of twisted complexes are allowed (and encouraged!)
## At the lowest level of iterations, things should be of class CFK() or CF()


class Twisted_Complex(object):
    def __init__(self,Obs,Mors):    
        self.Obs=Obs
        self.Mors=Mors
    # we return the total complex
    ##this will be a CFK() complex
    def Tot(self):
        if type(self.Obs[0])==Twisted_Complex:
            new_obs=[self.Obs[i].Tot() for i in range(len(self.Obs))]
            new_mors_list=[[matrix_to_mor(self.Mors.matrix[i][j]) for j in range(len(self.Obs))] for i in range(len(self.Obs))]
            Y=Twisted_Complex(new_obs,mor_matrix(new_mors_list))
            return Y.Tot()
        else:
            new_cx=self.Obs[0].copy()
            for i in range(1,len(self.Obs)):
                new_cx=new_cx+self.Obs[i]
            delta_1=matrix_to_mor(self.Mors)
            new_cx.diff=new_cx.diff+delta_1
            return new_cx
    ## assumes that self is (eventually) of type CFK
    ## shifts bigrading of lowest level of self
    def bi_gr_shift(self,shift):
        for X in self.Obs:
            X.bi_gr_shift(shift)

            
        #if type(self.Obs[0])==Twisted_Complex:
        #    self.Obs=[self.Obs[X].bi_gr_shift(shift) for X in self.Obs]
        #else:
        #    for X in self.Obs:
        #        X.bi_gr_shift(shift)
    
    
    
    
##################
###            ###
### mor_matrix ###
###            ###
##################
    
## the following is a class of matrices of morphisms
##unlike mor class,
## no need for co_rank input since matric contains the number of summands in domain and codomain
## these are natural morphisms of twisted complexes


class mor_matrix(object):
    def __init__(self,matrix): 
        self.matrix=matrix
        ## co_rank is the number of summands in the codomain
        self.co_rank=len(matrix)
        ## rank is the number of summands in the domain.
        self.rank=len(matrix[0])
        rank=self.rank
    # this adds two matrices
    def __add__(self, mor2):
        new_matrix=[[self.matrix[i][j]+mor2.matrix[i][j] for j in range(self.rank)] for i in range(self.co_rank)]
        return mor_matrix(new_matrix)
    ## The purpose of this method is to return the composition of the two maps.
    def __mul__(self,mor2):
        new_map=[]
        G=self.matrix
        F=mor2.matrix
        rank1=mor2.rank
        rank2=mor2.co_rank
        rank3=self.co_rank
        new_matrix=[[sum_list([G.matrix[i][k]*F.matrix[k][j] for k in range(rank2)]) for i in range(rank3)] for j in range(rank1)]
        return mor_matrix(new_matrix)
    
    ## here f is mor_matrix or mor from X_j to Y_i. We modify self by adding f to this component
    ## this changes self, but does not change f.
    
    def add_component(self,i,j,f):
        self.matrix[i][j]=self.matrix[i][j]+f
        
## The following functions return the zero mor matrices in several contexts
## gives the zero_mor_matrix
## returns a matrix of mors
## here ranks and co_ranks are lists of integers
## if just ranks is passed, then we make the output an endomorphism (Set co_ranks=ranks)

def zero_mor_matrix_ranks(ranks,co_ranks=-1):
    if co_ranks==-1:
        co_ranks=ranks
    Mors=[[set() for j in range(len(ranks))] for i in range(len(co_ranks))]
    for i in range(len(co_ranks)):
        for j in range(len(ranks)):
            Mors[i][j]=zero_mor(ranks[j],co_ranks[i])
    return mor_matrix(Mors)




## this is similar to the above, except now we have input equal to a pair of twisted complexes
## this method is recursive, and can give a morphism of twisted complexes of... of twisted complexes
##(as many iterations) as needed

def zero_mor_matrix_Tw(Tw_Dom,Tw_Cod):
    if type(Tw_Dom.Obs[0])==Twisted_Complex:
        return mor_matrix([[zero_mor_matrix_Tw(X,Y) for X in Tw_Dom.Obs] for Y in Tw_Cod.Obs])
    else:
        ranks=[len(X.gens) for X in Tw_Dom.Obs]
        co_ranks=[len(Y.gens) for Y in Tw_Cod.Obs]
        return zero_mor_matrix_ranks(ranks,co_ranks)
    


## transforms a mor_matrix object to a mor object on X_1+...+X_n
def matrix_to_mor(M):
    ## this is a backtracking function
    ## firstly, matrices of matrices are converted into matrices via backtracking
    if type(M.matrix[0][0])==mor_matrix:
        new_matrix=[[matrix_to_mor(M.matrix[i][j]) for j in range(M.rank)] for i in range(M.co_rank)]
        return matrix_to_mor(mor_matrix(new_matrix))
    ## the final step is the case when the type of M.matrix[0][0] is mor
    else:
        ##cumulative_rank[i] is the rank of X_0+...+X_{i-1}
        cumulative_rank=[0]
        for i in range(0,M.rank):
            rank=cumulative_rank[i]+M.matrix[0][i].rank
            cumulative_rank.append(rank)
        rank=cumulative_rank[M.rank]
        ##similar to cumilative_rank
        cumulative_co_rank=[0]
        for i in range(0,M.co_rank):
            co_rank=cumulative_co_rank[i]+M.matrix[i][0].co_rank
            cumulative_co_rank.append(co_rank)
        co_rank=cumulative_co_rank[M.co_rank]
        new_mor=zero_mor(rank,co_rank)
        for i in range(M.co_rank):
            for j in range(M.rank):
                new_mor.add_block(M.matrix[i][j],cumulative_rank[j],cumulative_co_rank[i])
        return new_mor
        
        
    
    


In [4]:
##################
###            ###
###  Examples  ###
###            ###
##################


##input is a list of generators of filtered CFKinfty
##meaning [g,i,j] represtenting an element x u^i v^j with A=0 and Maslov grading 0
#output is a list of elements [gr_w,gr_z]

def filtered_to_graded(filt_gens):
    gr_gens=[]
    for x in filt_gens:
        gr_gens.append([x[0]+2*x[1],x[0]+2*x[2]])
    return gr_gens



#input is a list of elements of the form [gr_w,gr_z]
#output is a list of elements of the form [g,i,j] (actually [g,i,0], for simplicity)

def graded_to_filtered(gr_gens):
    filt_gens=[]
    for x in gr_gens:
        A=(1/2)*(x[0]-x[1])
        filt_gens.append([x[1],A,0])
    return filt_gens


from sympy.abc import T

#input is a polynomial (i.e. non-negative exponents)
#output is the CFK-I-staircase complex
## polynomials should be inputed via f=poly(x**2+x+1) etc.
def CFK_from_polynomial(poly):
    exp=list(reversed(poly.monoms()))
    exp=[x[0] for x in exp]
    #n is the number of generators
    n = len(exp)
    degree = exp[n-1]
    ds = [exp[i+1] - exp[i] for i in range(n-1)]
    #i_list is the list of i-gradings: [degree, degree - d[0], degree - d[0], degree - d[0] + d[2], degree - d[0] + d[2], ...]
    i_list = [degree//2]
    for k in range(n-1):
        if k%2 == 0:
            i_list += [i_list[k] - ds[k], i_list[k] - ds[k]]  
    #Diff is the list of differentials of the generators
    Diff = []
    for k in range(n):
        if k%2 == 0:
            Diff.append(set())
        else:
            Diff += [{k-1, k+1}]
    gen = [[-degree + k%2, i_list[k], i_list[n-k-1]] for k in range(n)]
    return CFK(filtered_to_graded(gen), endo(Diff))



def Alexander(p,q):
    numerator=poly((T**(p*q)-1)*(T-1))
    denominator=poly((T**p-1)*(T**q-1)) 
    return div(numerator,denominator)[0]


def CFK_Torus(p,q):
    f=Alexander(p,q)
    return CFK_from_polynomial(f)

def XK_Torus(p,q):
    return CFK_to_XK(CFK_Torus(p,q))



## The following returns a complex with a single tower generator, and an n x n box.
##Box_n(1) is the complex of the figure 8.

def Box_n(n):
    Boxn_gens=[[0,0,0],[0,0,0],[-1,n,0],[-1,0,n],[-2,n,n]]
    Boxn_diff=[set(),{2,3},{4},{4},set()]
    return CFK(filtered_to_graded(Boxn_gens), endo(Boxn_diff))


## returns trefoil complex with length n legs
def long_trefoil(n):
    gens=[[-2*n,0],[-2*n+1,-2*n+1],[0,-2*n]]
    diff=endo([{},{0,2},{}])
    return CFK(gens,diff)




In [5]:
###
###
### This cell contains some homological and algebraic functions
### These include adding maps, composing maps and finding null-homotopies
###
###
###






                
#The following is a helper function.
#We assume E is a map from C to C (i.e. an endomorphism)
##which is a list of sets
#Input is old map (e.g. the differential)
#outputs map in new basis (e.g. new differential)
#note, this changes the old map E
#We assume that x_i and x_j have the same parity
#We assume g(x_j)>g(x_i)
#The change of basis is that x_i is replaced by x_i+U^k x_j
#x_j is left the same


def basis_change(E,i,j):
    #first changes E summands going to i 
    for Ex in E:
        if i in Ex:
            if j not in Ex:
                Ex.add(j)
            else:
                Ex.remove(j)
    #then adds terms going from j to those going from i
    for y in E[j]:
        if y not in E[i]:
            E[i].add(y)
        else:
            E[i].remove(y)
    return E


#The following is a helper function.
#The change of basis is that x_i is replaced by x_i+U^k x_j
#x_j is left the same
##It is a (hopefully faster) version of the above
##We assume E_dual is precomputed (so we don't have to go through E)
#changes both E and E_dual
##Note input is different than for "basis_change" since we need E_dual precomputed.
 ###New version           
def basis_change_efficient(E,E_dual,i,j):
    for k in E_dual[i]:
        if j in E[k]:
            E[k].remove(j)
        else:
            E[k].add(j)
        if k in E_dual[j]:
            E_dual[j].remove(k)
        else:
            E_dual[j].add(k)
    for k in E[j]:
        if k in E[i]:
            E[i].remove(k)
        else: 
            E[i].add(k)
        if i in E_dual[k]:
            E_dual[k].remove(i)
        else:
            E_dual[k].add(i)     

            
            
        
    ##These are similar to basis_change, except that we only perform the basis change to the domain.
    #both E and E_dual are modified.
    # we always assume that i\neq j
def basis_change_domain(E,E_dual,i,j):
    for k in E[j]:
        if k in E[i]:
            E[i].remove(k)
            E_dual[k].remove(i)
        else:
            E[i].add(k)
            E_dual[k].add(i)
    
    ##These are similar to basis_change, except that we only perform the basis change to the domain.
    #Both E and E_dual are modified
def basis_change_codomain(E,E_dual,i,j):
    for k in E_dual[i]:
        if j in E[k]:
            E[k].remove(j)
            E_dual[j].remove(k)
        else:
            E[k].add(j)
            E_dual[j].add(k)
            

#if E is a list endomorphism
#we write E in the basis where we have swapped x_i and x_j.
#note, this changes E.
###WARNING, don't forget to also swap the elements of gens if you are using this method.

def basis_swap(E,i,j):
    Ei=E[i]
    Ej=E[j]
    E[j]=Ei
    E[i]=Ej
    for x in range(len(E)):
        if i in E[x] and j not in E[x]:
            E[x].remove(i)
            E[x].add(i)
        if i not in E[x] and j in E[x]:
            E[x].remove(j)
            E[x].add(i)



            
### We now build CF_reduced_SDR
## This complex has input a CF complex C
## The output will be a tuple [C_red,Pi,I,H]
## Where C_red is reduced and Pi, I, H are a strong deformation retraction from C to C_red
            
def CF_reduced_SDR(C):
    rank=C.rank
    gens=[i for i in C.gens]
    ##diff and diff dual are just lists (i.e. not of class mor)
    diff=[{i for i in x} for x in C.diff.matrix]
    ##
    diff_dual=C.diff.dual().matrix
    ###
    ### F_map will be the map from C to C_basis_change
    ### G_map will be the inverse map.
    ### F_map and G_map are lists (as opposed to mors)
    ##
    F_map=identity_map(rank)
    F_map_dual=identity_map(rank)
    G_map=identity_map(rank)
    G_map_dual=identity_map(rank)
    ###
    ###
    to_delete=[0 for i in range(rank)]
    shift_fn=[0 for i in range(rank)]
    #H will be the homotopy for homological perturbation lemma
    H0=[set() for i in range(rank)]
    for x in range(rank):
        known_short_arrow=false
        y_special=0
        for y in diff[x]:
            if (gens[y]-gens[x]+1)/2==0:
                known_short_arrow=true
                y_special=y
                break
        if known_short_arrow:
            #move y_special to the end of diff[x]
            diff_x_non_special={i for i in diff[x] if i!=y_special}
            for y in diff_x_non_special:
                basis_change_efficient(diff,diff_dual, y_special,y)
                basis_change_codomain(F_map,F_map_dual, y_special,y)
                basis_change_domain(G_map,G_map_dual, y_special,y)
            other_xs=[xo for xo in diff_dual[y_special]]
            for x_other in other_xs:
                if x_other!=x:
                    basis_change_efficient(diff,diff_dual,  x_other,x)
                    basis_change_codomain(F_map,F_map_dual, x_other,x)
                    basis_change_domain(G_map,G_map_dual, x_other,x)
            to_delete[y_special]=1
            to_delete[x]=1
            H0[y_special].add(x)
    shift_fn[0]=to_delete[0]
    ## We will now build maps Pi and I
    I0=mor([{x} for x in range(rank) if to_delete[x]==0],rank)
    H0=mor(H0,rank)
    Pi0=dual_mor(I0)
    G_mor=mor(G_map,rank)
    F_mor=mor(F_map,rank)
    I=G_mor*I0 
    Pi=Pi0*F_mor
    H=G_mor*H0*F_mor
    ##
    for j in reversed(range(rank)):
        if to_delete[j]==1:
            del gens[j]
    diff_new=Pi0*endo(diff)*I0 
    return [CF(gens,diff_new),Pi,I,H]



### We now build CFK_reduced_SDR
## This complex has input a CFK complex C
## The output will be a tuple [C_red,Pi,I,H]
## Where C_red is reduced and Pi, I, H are a strong deformation retraction from C to C_red
            
def CFK_reduced_SDR(C):
    rank=C.rank
    gens=[i for i in C.gens]
    diff=[{i for i in x} for x in C.diff.matrix]
    ##
    diff_dual=C.diff.dual().matrix
    ###
    ### F_map will be the map from C to C_basis_change
    ### G_map will be the inverse map.
    ###
    F_map=identity_map(rank)
    F_map_dual=identity_map(rank)
    G_map=identity_map(rank)
    G_map_dual=identity_map(rank)
    ###
    ###
    to_delete=[0 for i in range(rank)]
    shift_fn=[0 for i in range(rank)]
    #H will be the homotopy for homological perturbation lemma
    H0=[set() for i in range(rank)]
    for x in range(rank):
        known_short_arrow=false
        y_special=0
        for y in diff[x]:
            if  ((gens[y][0]-gens[x][0]+1)/2==0 and (gens[y][1]-gens[x][1]+1)/2==0):
                known_short_arrow=true
                y_special=y
                break
        if known_short_arrow:
            #move y_special to the end of diff[x]
            diff_x_non_special={i for i in diff[x] if i!=y_special}
            for y in diff_x_non_special:
                basis_change_efficient(diff,diff_dual, y_special,y)
                basis_change_codomain(F_map,F_map_dual, y_special,y)
                basis_change_domain(G_map,G_map_dual, y_special,y)
            other_xs=[xo for xo in diff_dual[y_special]]
            for x_other in other_xs:
                if x_other!=x:
                    basis_change_efficient(diff,diff_dual,  x_other,x)
                    basis_change_codomain(F_map,F_map_dual, x_other,x)
                    basis_change_domain(G_map,G_map_dual, x_other,x)
            to_delete[y_special]=1
            to_delete[x]=1
            H0[y_special].add(x)
    shift_fn[0]=to_delete[0]
    ## We will now build maps Pi and I
    I0=mor([{x} for x in range(rank) if to_delete[x]==0],rank)
    H0=mor(H0,rank)
    Pi0=dual_mor(I0)
    G_mor=mor(G_map,rank)
    F_mor=mor(F_map,rank)
    I=G_mor*I0 
    Pi=Pi0*F_mor
    H=G_mor*H0*F_mor
    ##
    for j in reversed(range(rank)):
        if to_delete[j]==1:
            del gens[j]
    diff_new=Pi0*endo(diff)*I0 
    return [CFK(gens,diff_new),Pi,I,H]


## same as above except does not return the extra maps
def CFK_reduced(C):
    return CFK_reduced_SDR(C)[0]

### We now build CFT_reduced_SDR
## This complex has input a CFT complex C
## The output will be a tuple [C_red,Pi,I,H]
## Where C_red is reduced and Pi, I, H are a strong deformation retraction from C to C_red
            
def CFT_reduced_SDR(C):
    rank=C.rank
    gens=[i for i in C.gens]
    diff=[{i for i in x} for x in C.diff.matrix]
    ##
    diff_dual=C.diff.dual().matrix
    ###
    ### F_map will be the map from C to C_basis_change
    ### G_map will be the inverse map.
    ###
    F_map=identity_map(rank)
    F_map_dual=identity_map(rank)
    G_map=identity_map(rank)
    G_map_dual=identity_map(rank)
    ###
    ###
    to_delete=[0 for i in range(rank)]
    shift_fn=[0 for i in range(rank)]
    #H will be the homotopy for homological perturbation lemma
    H0=[set() for i in range(rank)]
    for x in range(rank):
        known_short_arrow=false
        y_special=0
        for y in diff[x]:
            if  (gens[y][0]-gens[x][0]+1)/2==0:
                known_short_arrow=true
                y_special=y
                break
        if known_short_arrow:
            #move y_special to the end of diff[x]
            diff_x_non_special={i for i in diff[x] if i!=y_special}
            for y in diff_x_non_special:
                basis_change_efficient(diff,diff_dual, y_special,y)
                basis_change_codomain(F_map,F_map_dual, y_special,y)
                basis_change_domain(G_map,G_map_dual, y_special,y)
            other_xs=[xo for xo in diff_dual[y_special]]
            for x_other in other_xs:
                if x_other!=x:
                    basis_change_efficient(diff,diff_dual,  x_other,x)
                    basis_change_codomain(F_map,F_map_dual, x_other,x)
                    basis_change_domain(G_map,G_map_dual, x_other,x)
            to_delete[y_special]=1
            to_delete[x]=1
            H0[y_special].add(x)
    shift_fn[0]=to_delete[0]
    ## We will now build maps Pi and I
    I0=mor([{x} for x in range(rank) if to_delete[x]==0],rank)
    H0=mor(H0,rank)
    Pi0=dual_mor(I0)
    G_mor=mor(G_map,rank)
    F_mor=mor(F_map,rank)
    I=G_mor*I0 
    Pi=Pi0*F_mor
    H=G_mor*H0*F_mor
    ##
    for j in reversed(range(rank)):
        if to_delete[j]==1:
            del gens[j]
    diff_new=Pi0*endo(diff)*I0 
    return [CFT(gens,diff_new),Pi,I,H]










##This method takes an XK_complex and produces a reduced version.
## The algorithm is to apply CFK_reduced_SDR to idempotent 1 and CFT_reduced_SDR to idempotent 1.

    
def XK_reduced(X):
    C0_SDR=CFK_reduced_SDR(X.CFK())
    C1_SDR=CFT_reduced_SDR(X.CFT())
    C0=C0_SDR[0]
    Pi_0=C0_SDR[1]
    I_0=C0_SDR[2]
    H_0=C0_SDR[3]
    C1=C1_SDR[0]
    Pi_1=C1_SDR[1]
    I_1=C1_SDR[2]
    H_1=C1_SDR[3]
    v_map_new=Pi_1*X.v_map*I_0
    h_map_new=Pi_1*X.h_map*I_0
    return XK(C0.gens,C1.gens,C0.diff,C1.diff,v_map_new,h_map_new)
    
    
    
    ##Here X_1 and X_2 are two XKI complexes
    ## Output is their tensor product. We are going to use the diagram shown in Figure 1.1 of "framing.pdf", which
    ## is given by "model 3"
def XK_tensor(X1,X2):
    X1_0=X1.CFK()
    X1_1=X1.CFT()
    X2_0=X2.CFK()
    X2_1=X2.CFT()
    T_0=X1_0*X2_0
    T_1=X1_1*X2_1
    ## Build the v_map and h_map
    
    new_h_map=tensor(X1.h_map,X2.h_map)
    new_v_map=tensor(X1.v_map,X2.v_map)
    return XK(T_0.gens,T_1.gens,T_0.diff,T_1.diff,new_v_map, new_h_map)



### The following takes a morphism "f" and expands both the domain and codomain
### shift_dom is where the indices for f are to begin (0 based indexing of course),
###and new_dom is the total size of the new_domain
### shift_cod is where the indices for f are to begin and new_cod is the total size of the new codomain

def shift_mor(f,shift_dom, shift_cod, new_dom, new_cod):
    F_matrix=[set() for i in range(shift_dom)]
    F_matrix.extend([{i+shift_cod for i in x} for x in f.matrix])
    for j in range(len(F_matrix),new_dom):
        F_matrix.append(set())
    return mor(F_matrix,new_cod)



## returns A_s
## viewed as a subcomplex of CFK(K)
## equipped with the gr_w grading.
## it is an object of class CF

def A_s(X,s):
    new_gens=[]
    for x in X.gens_0:
        if (x[0]-x[1])/2 <=s:
            #in this case, the generator is Z^{s-A(x)}x
            new_gens.append(x[0])
        if (x[0]-x[1])/2>s:
            # in this case, the generator is W^{A(x)-s} x
            new_gens.append(x[0]-2*((x[0]-x[1])*Rational(1/2)-s))
    diff_matrix=[{i for i in x} for x in X.diff_0.matrix]
    A_s=CF(new_gens,endo(diff_matrix))
    return A_s


## Same as above except it returns the B_s complex. (basically just forgets about one of the gradings)
def B_s(X,s):
    C=[x[0] for x in X.gens_1]
    diff_matrix=[{i for i in x} for x in X.diff_1.matrix]
    return CF(C,endo(diff_matrix))






def alex(x):
    return (x[0]-x[1])*Rational(1/2)





##input is a CF_old complex (where differential is a list of sets instead of a mor)
##output is a list of the correction terms.
##only works for CF complexes with one or two towers

def d_list(C):
    rank=C.rank
    gens=[i for i in C.gens]
    diff=[{i for i in x} for x in C.diff]
    C_dual=C.dual()
    diff_dual=[{i for i in x} for x in C_dual.diff]
    arrow_length=0
    #first delete arrows of length 0, then 1 etc. each time we go through while loop
    ## we increase arrow_length by 1, until diff "is" zero.
    unpaired_generators_set={i for i in range(rank)}
    unpaired_generators_num=rank
    while unpaired_generators_num>2:
        to_remove=[]
        for x in unpaired_generators_set:
            known_short_arrow=false
            y_special=0
            for y in diff[x]:
                if (gens[y]-gens[x]+1)/2==arrow_length:
                    known_short_arrow=true
                    y_special=y
                    break
            if known_short_arrow:
                diff_x_non_special=[i for i in diff[x] if i!=y_special]
                for y in diff_x_non_special:
                    basis_change_efficient(diff,diff_dual, y_special,y)
                ###We are here going to make an edit and see if it improves efficiency
                other_xs=[xo for xo in diff_dual[y_special]]
                for x_other in other_xs:
                    if x_other!=x:
                        basis_change_efficient(diff,diff_dual, x_other,x)
                to_remove.append(y_special)
                to_remove.append(x)
                unpaired_generators_num+=-2
        for x in to_remove:
            unpaired_generators_set.remove(x)
        arrow_length=arrow_length+1
#        print(arrow_length)
    new_gens=[gens[i] for i in unpaired_generators_set]
    return new_gens





In [6]:
##R= Rational['Z1,Z2']
##R.inject_variables()
##S=Rational['T']
##S.inject_variables()

##these are the variables for 2-component Alexander polynomials these are going to be t1^(1/2) and t2^(1/2)
## respectively, so that we don't need to deal with fractional exponents

from sympy.abc import U, V

## variable for single variable alexander poly
from sympy.abc import T

##################
###            ###
### KXR class  ###
###            ###
##################

##We now define the class KXR
##This is encoded as follows
##   linking is the linking number of the pattern (which is the linking number of the two components of L_P
## s1_min is the minimum value of s1 that we have a complex for.
##   s_max should be a rational number Rational or an integer. (takes values in linking)
##   we assume that s_range is symmetric
##   Outside of the range s_range, Cs is either a shifted version of C_{min s} or C_{max s} 
##   Cs is a list of CFK complexes, one in each alexander grading.
##   S is a single CFK complex (idempotent 10 of the bimodule)
##   L_W (etc.) are lists of maps, corresponding to the various data of the bimodule
##   For L_W and L_Z we also include the data for the edge values of s (even if though they are determined and not interesting)
## the internal versions of the data are just lists of CFK complexes or maps
## the function versions have as input an s value, and the output is the corresponding map
## e.g. self.Cs(5/2) returns the complex C_{5/2}
## The s value input of all maps corresponds to the *domain* of the map

class KXR(object):
    def __init__(self,linking,s1_min,Cs,S,L_W,L_Z,h_WZ, h_ZW, L_sig,L_tau,h_sigW, h_sigZ, h_tauW,h_tauZ):
        self.linking=linking
        self.s1_min=s1_min
        self.s1_max=len(Cs)+s1_min-1
        self.Cs_list=Cs
        self.S_internal=S
        self.L_W_list=L_W
        self.L_Z_list=L_Z
        self.h_WZ_list=h_WZ
        self.h_ZW_list=h_ZW
        self.L_sig_list=L_sig
        self.L_tau_list=L_tau
        self.h_sigW_list=h_sigW
        self.h_sigZ_list=h_sigZ
        self.h_tauW_list=h_tauW
        self.h_tauZ_list=h_tauZ
        self.ranks=[C.rank for C in self.Cs_list]
    ## returns the rank of the complex Cs(s)
    def rank(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.ranks[s-self.s1_min]
        if s>self.s1_max:
            return self.ranks[self.s1_max-self.s1_min]
        if s<self.s1_min:
            return self.ranks[0]
    ## returns the Cs complex, which is a CFK complex
    ## s is in Z+lk/2. (int or Rational)
    ##Note, this returns a COPY of the complexes in the staircases (so no need to copy again).
    def S(self):
        return self.S_internal.copy()
    def Cs(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.Cs_list[s-self.s1_min].copy()
        if s>self.s1_max:
            Cmax=self.Cs_list[self.s1_max-self.s1_min].copy()
            bi_gr_shift(Cmax.gens,[0,-2*(s-self.s1_max)]) 
            return Cmax    
        if s<self.s1_min:
            Cmin=self.Cs_list[0].copy()
            bi_gr_shift(Cmin.gens,[-2*(self.s1_min-s),0])
            return Cmin
    def L_W(self,s):
        if s>=self.s1_min and s<=self.s1_max:
            return self.L_W_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return identity_mor(self.Cs_list[0].rank)
        if s>self.s1_max:
            return identity_mor(self.Cs_list[len(self.Cs_list)-1].rank)
    def L_Z(self,s):
        if s>=self.s1_min and s<=self.s1_max:
            return self.L_Z_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return identity_mor(self.Cs_list[0].rank)
        if s>self.s1_max:
            return identity_mor(self.Cs_list[len(self.Cs_list)-1].rank)
    def h_WZ(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_WZ_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank)
    def h_ZW(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_ZW_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank)
    def L_sig(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.L_sig_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return self.L_sig_list[0].copy()
        if s>self.s1_max:
            return self.L_sig_list[self.s1_max-self.s1_min].copy()
    def L_tau(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.L_tau_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return self.L_tau_list[0].copy()
        if s>self.s1_max:
            return self.L_tau_list[len(self.Cs_list)-1].copy()         
    def h_sigW(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_sigW_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank,self.S_internal.rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank,self.S_internal.rank)         
    def h_sigZ(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_sigZ_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank,self.S_internal.rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank,self.S_internal.rank) 
    def h_tauW(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_tauW_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank,self.S_internal.rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank,self.S_internal.rank)
    def h_tauZ(self,s):
        if self.s1_min<=s and s<=self.s1_max:
            return self.h_tauZ_list[s-self.s1_min].copy()
        if s<self.s1_min:
            return zero_mor(self.Cs_list[0].rank,self.S_internal.rank)
        if s>self.s1_max:
            return zero_mor(self.Cs_list[len(self.Cs_list)-1].rank,self.S_internal.rank)

            
#################
###           ###
### Staircase ###
###           ###
#################

## This class is very similar to CFK, except is more adapted
## to computations using staircase complexes
## Here gens_0 is a list of the alg 0 generators in the staircase
## gens_0 should be put with increasing gr_w.
## there is an optional argument (ordered)
## if passed as False, then we will order them
## gens_0 and gens_1 are the generators in algebraic grading 0 or 1
## gens is all generators (put in standard order for staircase, interleaving gens_0 and gens_1)
## diff is differential


class staircase(object):
    def __init__(self,gens_0,ordered=True):
        if not ordered:
            self.gens_0=sorted(gens_0, key=lambda x: x[0])
        self.gens_0=gens_0
        gens_1=[]
        for x in range(len(gens_0)-1):
            grw_x=gens_0[x][0]
            grz_x=gens_0[x][1]
            grw_y=gens_0[x+1][0]
            grz_y=gens_0[x+1][1]
            gens_1.append([grw_x+1,grz_y+1])
        self.gens_1=gens_1        
        gens=[gens_0[0]]
        diff_matrix=[set()]
        for x in range(1,len(gens_0)):
            gens.append(gens_1[x-1])
            gens.append(gens_0[x])
            diff_matrix.append({2*x-2,2*x})
            diff_matrix.append(set())
        self.gens=gens
        self.diff=endo(diff_matrix)
        self.rank_0=len(gens_0)
        self.rank_1=len(gens_1)
        self.rank=2*self.rank_0-1

    def copy(self):
        gens_0_copy=[[x for x in y] for y in self.gens_0]
        return staircase(gens_0_copy)
    
    def CFK(self):
        return CFK(self.gens, self.diff)            
            
###############
###        ####
### h_func ####
###        ####
###############

## this is a class which encodes the h_function of a knot (or a slice of the h-function of a link)
## the input consists of a decreasing list of integers h_list
## and an integer s_min. Here s_min is the Alexander grading of the smallest entry in h_list
## The class encodes a function h so that
## h(s_min+s)=h_list[s] for s in {0,..., len(h_list)-1}
## we set h(s)=h(s_min)+(s_min-s) for s<s_min
## and h(s)=h(s_max) for s>s_max, where s_max=s_min+len(h_list)-1

class h_func(object):
    def __init__(self,h_list,s_min):
        self.h_list=h_list
        self.s_min=s_min
        self.s_max=s_min+len(h_list)-1
        
    ## the function ev produces the value of h(s)
    def ev(self, s):
        if s<self.s_min:
            return self.h_list[0]+self.s_min-s
        if s>self.s_max:
            return self.h_list[len(self.h_list)-1]
        return self.h_list[s-self.s_min]
    
    ##the following returns a staircase complex which has the given h-function
    
    def CFK(self):
        special_indices=[]
        for s in range(self.s_min,self.s_max+1):
            if self.ev(s)==self.ev(s+1) and self.ev(s-1)>self.ev(s):
                special_indices.append(s)
        gens=[]
        SI_ind_prev=special_indices[0]
        SI_len=len(special_indices)
        s_prev=s_min+SI_ind_prev
        prev_grw=-2*h_list[SI_ind_prev]
        prev_grz=prev_grw-2*s_prev    
        gens.append([prev_grw,prev_grz])
        diff_matrix=[set()]
        for i in range(1,SI_len):
            s_current=s_min+special_indices[i]
            current_grw=-2*self.ev(s_current)
            current_grz=current_grw-2*s_current        
            ##adds alg grading 1 generator:
            gens.append([min(prev_grw,current_grw)+1,min(prev_grz,current_grz)+1])
            gens.append([current_grw,current_grz])
            diff_matrix.append({2*i-2,2*i})
            diff_matrix.append(set())
            prev_grw=current_grw
            prev_grz=current_grz
        return CFK(gens, endo(diff_matrix))
    
    ##similar to above, except returns an object of class staircase
    def staircase(self):
        special_s=[]
        for s in range(self.s_min,self.s_max+1):
            if self.ev(s)==self.ev(s+1) and self.ev(s-1)>self.ev(s):
                special_s.append(s)
        gens_0=[]
        for s in special_s:
            grw=-2*self.ev(s)
            grz=grw-2*s
            gens_0.append([grw,grz])
        return staircase(gens_0)

    ## the following prints the h_function
    ## it prints the h_function and h[0] between two bars | |
    ## if no RANGE is given, we just do s_min through s_max
    ## if RANGE is specified then we print h[i] for all i in the closed interval containing those endpoints.
    ## if 0 is not in the range, then we start by printing |s_min=..| for the smallest s value
    def print(self,RANGE=[0,0]):
        if RANGE==[0,0]:
            RANGE[0]=self.s_min
            RANGE[1]=self.s_max
        if 0<RANGE[0] or RANGE[1]<0:
            print("|s_min="+str(RANGE[0])+"|", end=" ")
        for s in range(RANGE[0],RANGE[1]+1):
            if s==0:
                print("|"+str(self.ev(s))+"|", end=" ")
            else:
                print(str(self.ev(s)), end=" ")
#################
###          ####
### h2_func ####
###          ####
#################

##this is a function which encodes the h_function of a two component link (in an L-space)
## here h_list is a list of lists all of the same length
## s_min is a list of two elements:
## ordered so that h[s_1][s_2] corresponds to h((s_1,s_2)+s_min)
## we assume that each list in h_list has the same length
## note that s_min should be a pair of rational numbers (Rational) or ints.
## not floats!

class h2_func(object):
    def __init__(self,h_list,s_min):
        self.s1_min=s_min[0]
        self.s2_min=s_min[1]
        self.s1_max=self.s1_min+len(h_list)-1
        self.s2_max=self.s2_min+len(h_list[0])-1
        self.h_list=h_list
        self.s1_range=len(h_list)
        self.s2_range=len(h_list[0])
    ## evaluates h(s_1,s_2) where s=(s1,s2)

    def ev(self,s1,s2):
        shift1=0
        shift2=0
        if s1>self.s1_max:
            s1=self.s1_max
        if s1<self.s1_min:
            shift1=self.s1_min-s1
            s1=self.s1_min
        if s2>self.s2_max:
            s2=self.s2_max
        if s2<self.s2_min:
            shift2=self.s2_min-s2
            s2=self.s2_min
        return self.h_list[s1-self.s1_min][s2-self.s2_min]+shift1+shift2
    
    
    

    
    ## here s is a number in 1/2+Z
    ## the following returns the h-like function t\mapsto h_L(s,t+s)
    ## returns a class of type h_func
    
    
    def slice(self,s1):
        new_s_min=self.s2_min+s1
        h_slice_list=[self.ev(s1,self.s2_min+i) for i in range(self.s2_range)]
        return h_func(h_slice_list,new_s_min)


    ## this prints the h-function
    ## if supported in 1/2+Z, then we put a | or ---- between Alexander grading 0 terms
    ## if supported in Z, then we put |x| and two lines of --- to denote Alexander grading 0
    ## RANGE1 is[s1_min,s1_max]
    ## indicating the print range, and RANGE2 is similar
    ##If nothing is entered, we set s1_min=s2_min and s1_max=s2_max
    ## and these are min(self.s1_min,self.s2_min) and similarly for max
    ## okay if RANGE1 does not have the same residue mod Z as linking (has some idiot proofing)
    ## spacing gets wonky if large RANGES are entered (could easily be fixed but sort of pointless)
    def print(self, RANGE1=[0,0],RANGE2=[0,0]):
        s1_min=0
        s1_max=0
        s2_max=0
        s2_min=0
        if RANGE1==[0,0] or RANGE2==[0,0]:
            s1_min=min(self.s1_min,self.s2_min)
            s1_max=max(self.s1_max,self.s2_max)
            s2_min=s1_min
            s2_max=s1_max
        else:
            parity_linking=(self.s1_min*2)%2
            if parity_linking==0:
                s1_min=math.floor(RANGE1[0])
                s1_max=math.ceil(RANGE1[1])
                s2_min=math.floor(RANGE2[0])
                s2_max=math.ceil(RANGE2[1])
            if parity_linking==1:
                s1_min=min(math.floor(RANGE1[0])+Rational(1/2),-Rational(1/2))
                s1_max=max(math.ceil(RANGE1[1])-Rational(1/2),Rational(1/2))
                s2_min=min(math.floor(RANGE2[0])+Rational(1/2),-Rational(1/2))
                s2_max=max(math.ceil(RANGE2[1])-Rational(1/2),Rational(1/2))
        ## grid width is the max length of the h values along the bottom row
        ## so that we can draw the picture
        grid_width=max([len(str(self.ev(s1_min+i,s2_min))) for i in range(s1_max-s1_min+1)])+1
        for j in reversed(range(s2_max-s2_min+1)):
            if s2_min+j==0 or s2_min+j==Rational(-1/2):
                print("--",end="")
                for x in range(grid_width*(s1_max-s1_min+1)):
                    print("-", end="")
                print()
            for i in range(s1_max-s1_min+1):
                if s1_min+i==0 or s1_min+i==Rational(1/2):
                    print("|",end=" ")
                symbol=str(self.ev(s1_min+i,s2_min+j))
                symbol_len=len(symbol)
                for k in range(grid_width-symbol_len):
                    symbol+=" "
                print(symbol, end="")
                if s1_min+i==0:
                    print("|", end=" ")
            if s2_min+j==0:
                print()
                print("--",end="")
                for x in range(grid_width*(s1_max-s1_min+1)):
                    print("-", end="")
            print()
    


    







#################
###           ###
### h2_to_KXR ###
###           ###
#################



    
## A are staircases and x is a linear combination of elements in algebraic grading 0.
## here we find an element y in alg grading 1 such that d(y)=x
## here x is a set. (A set {a,b,c} means x is m_a*a+m_b*b+m_c*c where m_a, m_b,m_c are monomials)
## The output y is also a set.

## Note that x and y have entries in {0,..., rank(A)-1}.
## (In particular the indexing )

## Note: mathematically, since A_1\to A_0 is injective, if such a y exists, it is unique.
## Note also that the answer doesn't depend on the monomials m_a, m_b, m_c, since A_1 and A_0 are free
## we assume that x only has even inputs (if odd, then non-sensical output may occur)

def find_boundary(A,x):
    x_copy={z for z in x}
    y=set()
    while not (x_copy==set() or x_copy=={}):
        m=min(x_copy)
        x_copy=x_copy.symmetric_difference({m,m+2})
        y=y.symmetric_difference({m+1})
    return y
        
        
    
## this function has input two staircases A,B
## and a chain map f\colon A\to B
## here f is just a mor, given in the base gen for A and B
## (i.e. not just for A_0 and B_0)
## we assume that f preserves the algebraic grading and
## is null-homotopic. The output is a null-homotopy of f.

##Algorithm: If A=(d_A:A_1->A_0) and similar for B
## then algebraically it is sufficient to solve d_B\circ h=f|A_{0}
## where h maps A_0 to B_1.
## returns a Mor from Tot(A) to Tot(B)

def null_homotopy(A,B,f):
    h_matrix=[set() for i in range(A.rank)]
    for x in range(A.rank_0):
        y=find_boundary(B,f.matrix[2*x])
        h_matrix[2*x]=h_matrix[2*x].union(y)
    return mor(h_matrix,B.rank)



## find_map is a function for finding alg-grading preserving
## chain maps between staircase complexes
## input are two staircases A and B and a list gr=[g0,g1]
## of even integers.
## function will return 0 and print "there is no such map" if a map does not exist

##note as of now, we do not carefully check whether such a map exists.
## only prints if parity is wrong

def gr_to_map(A,B,gr):
    f=zero_mor(A.rank, B.rank)
    if gr[0]%2==1 or gr[1]%2==1:
        print("there is no such map")
        return f
    for x in range(A.rank_0):
        found_fx=False
        candidate_y=0
        while not found_fx:
            y_gen=B.gens[2*candidate_y]
            if y_gen[0]>=A.gens[2*x][0]+gr[0] and y_gen[1]>=A.gens[2*x][1]+gr[1]:
                found_fx=True
                f.add_arrow(2*x,2*candidate_y)
            else:
                candidate_y+=1
            if candidate_y>=B.rank_0:
                print("no such map")
    f0=f.copy()
    g=f0*A.diff
    for x in range(A.rank_1):
        f.matrix[2*x+1]=find_boundary(B,g.matrix[2*x+1])
    return f


###
###
### This function has input an h2_func class, and outputs a KXR class
###
###
### Note that the code computes the linking number lk(L_1,L_2) from
### the h function. This can be computed as follows
### Call an Alexander grading (s1,s2) special if it corresponds to a module generator of 
### C_{s1}
### Let (s1_min,a) and (s2_max,b) denote the highest special generators of C_{s1_min} and C_{s1_max}
### then b-a is the linking number.

def h2_to_KXR(h):
    s1_range=h.s1_range
    s1_max=h.s1_max
    Cs_staircases=[h.slice(h.s1_min+i).staircase() for i in range(h.s1_range)]
    ## now we build the L_W maps
    ## we start by adding L_W on the lowest alexander grading, which takes the form
    ## of the identity map on generators
    L_W=[identity_mor(Cs_staircases[0].rank)]
    for i in range(1,s1_range):
        L_W.append(gr_to_map(Cs_staircases[i],Cs_staircases[i-1], [-2,0]))
    ## we now build the maps L_Z
    L_Z=[]
    for i in range(s1_range-1):
        L_Z.append(gr_to_map(Cs_staircases[i],Cs_staircases[i+1],[0,-2]))
    L_Z.append(identity_mor(Cs_staircases[s1_range-1].rank))
    ## we now make h_WZ and h_ZW
    h_WZ=[]
    for i in range(s1_range-1):
        g=identity_mor(Cs_staircases[i].rank)+L_W[i+1]*L_Z[i]
        h_WZ.append(null_homotopy(Cs_staircases[i],Cs_staircases[i],g))
    ## We can always assume that h_WZ is zero on Alexander grading s1_max
    ## similarly we can assume that h_ZW is zero on Alex. grading s1_min
    h_WZ.append(zero_mor(Cs_staircases[s1_range-1].rank))
    h_ZW=[zero_mor(Cs_staircases[0].rank)]
    for i in range(1,s1_range):
        g=identity_mor(Cs_staircases[i].rank)+L_Z[i-1]*L_W[i]
        h_ZW.append(null_homotopy(Cs_staircases[i],Cs_staircases[i],g))
    ## we now make the S complex
    ## we do this by taking the complex Cs[0] and normalizing so that it has gradings like a knot in S3
    PS=Cs_staircases[0].copy()
    ## we now normalize PS to get S_staircase
    ## here we are using that the generators in the class staircase are in proper order
    grw_shift=-max(PS.gens_0[0][0],PS.gens_0[PS.rank_0-1][0])
    grz_shift=-max(PS.gens_0[0][1],PS.gens_0[PS.rank_0-1][1])
    for x in range(PS.rank_0):
        PS.gens_0[x][0]+= grw_shift
        PS.gens_0[x][1]+= grz_shift
    for x in range(PS.rank_1):
        PS.gens_1[x][0]+= grw_shift
        PS.gens_1[x][1]+= grz_shift
    
    ## We now compute the linking number. We describe mathematically how this is encoded above
    ##
    grw_min=Cs_staircases[0].gens_0[0][0]
    grz_min=Cs_staircases[0].gens_0[0][1]
    s2_special_min=(grw_min-grz_min)*Rational(1/2)-h.s1_min
    grw_max=Cs_staircases[s1_range-1].gens_0[0][0]
    grz_max=Cs_staircases[s1_range-1].gens_0[0][1]
    s2_special_max=(grw_max-grz_max)*Rational(1/2)-h.s1_max
    ##
    linking=s2_special_max-s2_special_min
    ##
    
    ## we now record the grading shifts of the maps L_sig and L_tau
    ## we record the justification here to be able to double check
    ## L_sig: This map preserves gr_w and shifts drops A_2 by -lk/2   
    ## In I_0, we have gr_z=gr_w-2(A_1+A_2). In I_1, we have gr_z=gr_w-2A_2.
    ## 
    ##Therefore L_sig changes (gr_w,gr_z) by (0, 2A_1+linking)
    ##
    ## L_tau: This map preserves gr_z, and shifts A_2 by lk/2
    ## therefore the change in (gr_w,gr_z) is (-2A_1+linking,0).
    
    ## now we make L_sig and L_tau
    L_sig=[]
    L_tau=[]
    for t in range(s1_range):
        s=t+h.s1_min
        L_sig.append(gr_to_map(Cs_staircases[t],PS,[0,2*s+linking]))
        L_tau.append(gr_to_map(Cs_staircases[t],PS,[-2*s+linking,0]))
    ## we now compute the remaining homotopies h_sigW and so forth:
    h_sigW=[]
    h_sigZ=[]
    h_tauW=[]
    h_tauZ=[]       
    for t in range(s1_range):
        s=t+h.s1_min
        ##
        ##compute h_sigW
        UTsig_sigW=L_sig[t]+L_sig[max(t-1,0)]*L_W[t]
        h_sigW.append(null_homotopy(Cs_staircases[t],PS,UTsig_sigW))
        ##
        ## compute h_sigZ
        Tsig_sigZ=L_sig[t]+L_sig[min(s1_range-1,t+1)]*L_Z[t]
        h_sigZ.append(null_homotopy(Cs_staircases[t],PS,Tsig_sigZ))
        ##
        ## compute h_tauW
        Ttau_tauW=L_tau[t]+L_tau[max(t-1,0)]*L_W[t]
        h_tauW.append(null_homotopy(Cs_staircases[t],PS,Ttau_tauW))
        ##
        ## compute h_tauZ
        Ttau_tauZ=L_tau[t]+L_tau[min(s1_range-1,t+1)]*L_Z[t]
        h_tauZ.append(null_homotopy(Cs_staircases[t],PS,Ttau_tauZ))
        #
    Cs=[C.CFK() for C in Cs_staircases]
    S=PS.CFK()
    
    return KXR(linking,h.s1_min, Cs, S, L_W,L_Z,h_WZ,h_ZW,L_sig,L_tau,h_sigW,h_sigZ,h_tauW,h_tauZ)


    ## This function takes an Alexander polynomial for a knot and returns the h-function
    ## The alexander polynomial is normalized to be the Euler characteristic of HFK-hat
    ## we input the Alexander polynomial as a list of exponents
    ## the list does not need to be ordered, but there should be an odd number of elements in it
    ## I am not sure what happens if a list of non-integers is passed.
    ## for an L-space knot K, one should pass the terms of the symmetrized Alexander polynomial
    ## (non-symmetrized will yield a different h-function).
    ## the output is an object of class h_func.
    ## list doesn't have to be ordered (we sort it)
def Alex_to_h1(A):
    A_sorted=sorted(A,reverse=True)
    A_min=A_sorted[len(A_sorted)-1]
    A_max=A_sorted[0]
    s_min=A_min
    h_list=[0]
    gaps=[A_sorted[i]-A_sorted[i+1] for i in range(len(A_sorted)-1)]
    parity=0
    ## we first build the h_function in reverse
    h_list=[0]
    for i in range(0,len(A_sorted)-1):
        if parity==0:
            for j in range(gaps[i]):
                h_list.append(h_list[len(h_list)-1]+1)
        if parity==1:
            for j in range(gaps[i]):
                h_list.append(h_list[len(h_list)-1])
        parity=(parity+1)%2
    h_list.reverse()
    return h_func(h_list,s_min)
        
        
            
    

    



## the following function has input equal to 3 polynomials (input as tuples)
## A=Alexander poly of L
## A1=Alexander poly of L1=L-L2
## A2=Alexander poly of L2=L-L1
## the output is the h-function of L, using the Gorsky-Nemethi formula

##Note on normalizations: for A we use the multi-variable Alexander polynomial normalized as graded Euler characterist
## of HFL^- (Z=0 but W non-trivial). (This is a finite polynomial)
## For A_1 and A_2, we use the graded Euler characteristic of HFK-hat (also a finite polynomial).

## Note, we do not require A to be normalized by the correct monomial power,
## however we do require that it have the correct sign.


## the input of A is a list of triples
## [a,s1,s2]. Each a is an integer. Such an element means a t1^s1 t2^s2
## There can be repeated entries, etc.
## si should be either int or Rational
## now A1 and A2 are just lists of integers (like in Alexander_to_h1).
## for correct input, these should be symmetrized 
##(since L1 and L2 are L-space knots we know that the Alexander polys will be of the form t^{a_0}-t^{a_1}+t^{a_2}-...
## for some a_0<...<a_n where a_0=-a_n.
## we do not assume that the list is put in symmetric form, though we do normalize A1 and A2 so that this is the case.





def Alex_to_h2(linking,A,A1,A2):
    ## first we normalize A1 and A2
    A1_min=A1[0]
    A1_max=A1[len(A1)-1]
    A1_av=(A1_max+A1_min)*Rational(1/2)
    A1_normalized=[x-A1_av for x in A1]
    h1A1=Alex_to_h1(A1_normalized)

    A2_min=A2[0]
    A2_max=A2[len(A2)-1]
    A2_av=(A2_max+A2_min)*Rational(1/2)
    A2_normalized=[x-A2_av for x in A2]
    h1A2=Alex_to_h1(A2_normalized)
 

    #s1_min=min(h2L1_min,h2L2_pb.s1_min)
    s1_min=min(h1A1.s_min+linking*Rational(1/2)-1,0)
    s2_min=min(h1A2.s_min+linking*Rational(1/2)-1,0)
    for x in A:
        if x[1]<=s1_min:
            s1_min=x[1]-1
        if x[2]<=s2_min:
            s2_min=x[2]-1
    h_list=[]
    for i in range(-2*s1_min+1):
        s1=s1_min+i
        h_list_i=[]
        for j in range(-2*s2_min+1):
            s2=s2_min+j
            ## A_ij is the contribution to h_list[i,j] from the Alexander polynomial A
            A_ij=0
            for m in A:
                if m[1]>s1 and m[2]>s2:
                    A_ij+=m[0]
            h_list_i.append(h1A1.ev(s1-linking*Rational(1/2))+h1A2.ev(s2-linking*Rational(1/2))-A_ij)
        h_list.append(h_list_i)
    return h2_func(h_list,[s1_min,s2_min])


## we now have some functions which do the above but allow polynomial inputs.
## we first have a function which takes a polynomial in variables U and V,
## and returns a list corresponding to the multi-variate Alexander polynomial
## only nonnegative powers of U and V are allowed.




## we now have a function which normalizes the 2-variable Alexander polynomial (up to a sign)
## we normalize it by the convention that f(t1^{-1},t2^{-1})=t1^{-1}t2^{-1}f(t1,t2)
## we normalize the sign so that the returned function is a monomial (with coefficient 1) times f
## the output is a list of tuples of the form [a,i,j]

def normalize_A2(f):
    ## first check if f==0
    if f==0:
        return f
    shift=[0,0]
    # what we will do is record first the tuples of f as [a,i,j]
    ## then we will shift so that
    ##f(V^{-1},U^{-1})=U^{-2}V^{-2}f(U,V)
    ## then we multiply each i,j by Rational(1/2)

    ##listifying f
    if f==1:
        powers=[[0,0]]
    else:
        powers=[list(x) for x in f.monoms()]
    coeffs=[]
    if f==1:
        coeffs=[1]
    else:
        coeffs=f.coeffs()
    f_list=[[coeffs[x]]+powers[x] for x in range(len(powers))]

    ## normalizing:
    U1min=min([x[0] for x in powers])
    U1max=max([x[0] for x in powers])
    U2min=min([x[1] for x in powers])
    U2max=max([x[1] for x in powers])

    ## solving -(max+shift)=min+shift-2 gives the following
    shift1=(-U1max-U1min+2)//2
    shift2=(-U2max-U2min+2)//2

    return [[x[0],(x[1]+shift1)*Rational(1/2), (x[2]+shift2)*Rational(1/2)] for x in f_list]
    


## these polynomials should take the following form
## for A, we put this in a polynomial in U and V.
## Note:
##
##U=t1^(1/2) and V=t2^(1/2)
##
## We do this to avoid fractional exponents
## Now A1 and A2 are input as polynomials in T.
##(The signs of the coefficients of A1 and A2 are not important; we only read off the exponents)

def Alex_to_h2_poly(linking,A,A1,A2):
    A1_tuple=[x[0] for x in A1.monoms()]
    A2_tuple=[x[0] for x in A2.monoms()]
    A_tuple=normalize_A2(A)
    return Alex_to_h2(linking,A_tuple,A1_tuple,A2_tuple)





            
                

## We now build the code that's necessary to compute CFK(P_n(K))
## Steps:
## 1. Build the complexes E, F, J, M
## 2. Compute the maps phi^pmK, phi^pm mu and then the diagonal maps
## 3. Assemble complex for P_n(K)

## we now build Est
## K is an object of type XK
## L_P is an object of type KXR
#### the output is a twisted complex of CFK()'s###

## here s1 takes values in Z+1/2, and s2 takes values in Z+(1+w)/2
## type of s1, s2 should be Rational or int
##this returns a pair a twisted complex 


def Est(K,L_P,s,t):
    ## we begin by building a list called cx_list
    ## Est is a sum of staircase complexes, one for each generator of K.gens_0
    ## cx_list is a list of these staircase complexes (though they are of type CFK)
    cx_list=[]
    ## for convenience, we also make a list Alex_list where Alex_list[i] is Alexander grading of x_i
    ##
    ## sC_list is a list of the values of indices of the C staircases
    ## so sC_list[i]=t means that x_i\otimes C_t appears
    sC_list=[]
    sE_list=[]
    for i in range(len(K.gens_0)):
        A_i=(K.gens_0[i][0]-K.gens_0[i][1])//2
        sE_i=s-A_i
        sE_list.append(sE_i)
        ## here we are using x\otimes E_{s1,t}
        sC_i=0
        ## sC will be the actual index appearing, i.e. E_{sE_i,t}=C_{sC_i}
        if sE_i<0:
            sC_i=t-Rational(1/2)
        if sE_i>0:
            sC_i=t+Rational(1/2)
        Cx=L_P.Cs(sC_i)
        sC_list.append(sC_i)
        if sE_i<0:
            bi_gr_shift(Cx.gens, [2*sE_i+1+K.gens_0[i][0],K.gens_0[i][1]])
        if sE_i>0:
            bi_gr_shift(Cx.gens,[K.gens_0[i][0],1-2*sE_i+K.gens_0[i][1]])
        cx_list.append(Cx)
    ## now we have to build the twisted complex differential delta_1
    ## this does not include the self morphisms from the differentials in cx_list above
    ##
    ## we start by initializing delta_1 to be zero
    delta_1=zero_mor_matrix_ranks([C.rank for C in cx_list])
    ##
    ## we now start adding differentials. There are first terms corresponding to a single delta_1 from X(K):
    for x in range(len(K.gens_0)):
        for y in K.diff_0.matrix[x]:
            ## we use the values of sC_i to determine the map
            if sC_list[y]-sC_list[x]==0:
                delta_1.add_component(y,x,identity_mor(cx_list[x].rank))
            if sC_list[y]-sC_list[x]==1:
                ## we now want to use L_Z
                delta_1.add_component(y,x,L_P.L_Z(sC_list[x]))
            if sC_list[y]-sC_list[x]==-1:
                delta_1.add_component(y,x,L_P.L_W(sC_list[x]))
                ## the morphism is now L_W
    ##
    ## we now add the terms involving delta_3^1
    for x in range(len(K.gens_0)):
        for y in K.diff_0.matrix[x]:
            Delta_yx=sC_list[y]-sC_list[x]
            for z in K.diff_0.matrix[y]:
                Delta_zy=sC_list[z]-sC_list[x]
                ## we are going to add one of hwz or hzw to delta^1, under certain cirumstances.
                ## if Delta_yx and Delta_zy are both non-zero, then we will add either h_wz or hzw to the delta^1
                ## depending on signs.
                if Delta_yx==1 and Delta_zy==-1:
                    delta_1.add_component(z,x,L_P.h_WZ(sC_list[x]))
                if Delta_yx==-1 and Delta_zy==1:
                    delta_1.add_component(z,x,L_P.h_ZW(sC_list[x]))
    return Twisted_Complex(cx_list,delta_1)
    
## the following has input an XK complex
## and the  returns the list sC_list computed in the previous function
## for E_st

def E_sC_list(K,s,t):    
    sC_list=[]
    for i in range(len(K.gens_0)):
        A_i=(K.gens_0[i][0]-K.gens_0[i][1])/2
        sE_i=s-A_i
        sC_i=0
        ## sC will be the actual index appearing, i.e. E_{sE_i,t}=C_{sC_i}
        if sE_i<0:
            sC_i=t-Rational(1/2)
        if sE_i>0:
            sC_i=t+Rational(1/2)
        sC_list.append(sC_i)
    return sC_list
    ## now we have to build 
    
    ## we now build the twisted complex Fst
    ## this returns just a twisted cx
    
def Fst(K,L_P,s,t):
    cx_list=[]
    for i in range(len(K.gens_0)):
        A_i=(K.gens_0[i][0]-K.gens_0[i][1])*Rational(1/2)
        sF_i=s-A_i
        ## here we are using x_i\otimes F_{sF_i,t}
        Cx=L_P.S()
        bi_gr_shift(Cx.gens,[min(sF_i*2+1,0)+K.gens_0[i][0],min(0,-2*sF_i-1)+K.gens_0[i][1]])
        cx_list.append(Cx)
    delta_1=zero_mor_matrix_ranks([C.rank for C in cx_list])
    ## we now need to add differentials to delta_1
    ## note that there are no higher differentials,
    ## and we just add the identity from Cx to Cy if there is
    ## an arrow in K.diff_0 from x to y.
    for x in range(len(K.gens_0)):
        for y in K.diff_0.matrix[x]:
            delta_1.add_component(y,x,identity_mor(L_P.S_internal.rank))
    return Twisted_Complex(cx_list,delta_1)


##returns just a twisted cx
def Jst(K,L_P,s,t):
    cx_list=[L_P.Cs(t+Rational(1/2)) for x in range(len(K.gens_1))]
    for x in range(len(K.gens_1)):
        bi_gr_shift(cx_list[x].gens,[K.gens_1[x][0],K.gens_1[x][0]])
    C_rank=cx_list[0].rank
    delta_1=zero_mor_matrix_ranks([C_rank for C in cx_list])
    for x in range(len(K.gens_1)):
        for y in K.diff_1.matrix[x]:
            delta_1.add_component(y,x,identity_mor(C_rank))
    return Twisted_Complex(cx_list,delta_1)
    
    
## returns just a twisted cx.
def Mst(K,L_P,s,t):
    cx_list=[L_P.S() for x in range(len(K.gens_1))]
    for x in range(len(K.gens_1)):
        bi_gr_shift(cx_list[x].gens,[K.gens_1[x][0],K.gens_1[x][0]])    
    C_rank=cx_list[0].rank
    delta_1=zero_mor_matrix_ranks([C_rank for C in cx_list])
    for x in range(len(K.gens_1)):
        for y in K.diff_1.matrix[x]:
            delta_1.add_component(y,x,identity_mor(C_rank))
    return Twisted_Complex(cx_list,delta_1)

##computes the map from E_st to J_st
## which is phi_K.
def phi_K_EJ(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.rank(t+Rational(1/2)) for x in range(len(K.gens_1))]
    phi_K=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        for y in K.v_map.matrix[x]:
            if sC_list[x]<t:
                ## the map is L_Z
                phi_K.add_component(y,x,L_P.L_Z(t-Rational(1/2)))
            if sC_list[x]>t:
                ## the arrow is id
                phi_K.add_component(y,x,identity_mor(L_P.rank(t+Rational(1/2))))
    ## we now add the (sigma,W) term
    ## the rule is that we count sequences x-d->y-v->z
    ##where x has sC[x]>t while y has xC[y]<t
    for x in range(len(K.gens_0)):
        for y in K.diff_0.matrix[x]:
            if sC_list[x]>sC_list[y]:
                for z in K.v_map.matrix[y]:
                    phi_K.add_component(z,x,L_P.h_ZW(t+Rational(1/2)))
    return(phi_K)

def phi_mK_EJ(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.rank(t-Rational(1/2)) for x in range(len(K.gens_1))]
    phi_mK=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        for y in K.h_map.matrix[x]:
            if sC_list[x]>t:
                ## the map is L_W
                phi_mK.add_component(y,x,L_P.L_W(t+Rational(1/2)))
            if sC_list[x]<t:
                ## the arrow is id
                phi_mK.add_component(y,x,identity_mor(L_P.rank(t-Rational(1/2))))
    ## we now add the (tau,Z) term
    ## the rule is that we count sequences x-d->y-h->z
    ##where x has sC[x]<t while y has xC[y]>t
    for x in range(len(K.gens_0)):
        for y in K.diff_0.matrix[x]:
            if sC_list[x]<sC_list[y]:
                for z in K.h_map.matrix[y]:
                    phi_mK.add_component(z,x,L_P.h_WZ(t-Rational(1/2)))
    return(phi_mK)




# now we make phi^K from F to M
def phi_K_FM(K,L_P,s,t):
    ranks=[L_P.S_internal.rank for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi_K=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        for y in K.v_map.matrix[x]:
            phi_K.add_component(y,x,identity_mor(L_P.S_internal.rank))
    return phi_K


# now we make phi^mK from F to M
def phi_mK_FM(K,L_P,s,t):
    ranks=[L_P.S_internal.rank for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi_mK=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        for y in K.h_map.matrix[x]:
            phi_mK.add_component(y,x,identity_mor(L_P.S_internal.rank))
    return phi_mK


    ## now we make the f^mu maps from E to F

def phi_mu_EF(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_0))]
    phi_mu=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        phi_mu.add_component(x,x,L_P.L_sig(sC_list[x]))
        for y in K.diff_0.matrix[x]:
            if sC_list[x]>sC_list[y]:
                phi_mu.add_component(y,x,L_P.h_sigW(sC_list[x]))
            if sC_list[x]<sC_list[y]:
                phi_mu.add_component(y,x,L_P.h_sigZ(sC_list[x]))
    return phi_mu

## now we make f^{-mu} from E to F

def phi_mmu_EF(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_0))]
    phi_mmu=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        phi_mmu.add_component(x,x,L_P.L_tau(sC_list[x]))
        for y in K.diff_0.matrix[x]:
            if sC_list[x]>sC_list[y]:
                phi_mmu.add_component(y,x,L_P.h_tauW(sC_list[x]))
            if sC_list[x]<sC_list[y]:
                phi_mmu.add_component(y,x,L_P.h_tauZ(sC_list[x]))
    return phi_mmu


## now we make the f^mu map from J to M

def phi_mu_JM(K,L_P,s,t):
    ranks=[L_P.rank(t+Rational(1/2)) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_0))]    
    phi_mu=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        phi_mu.add_component(x,x,L_P.L_sig(t+Rational(1/2)))
    return phi_mu


def phi_mmu_JM(K,L_P,s,t):
    ranks=[L_P.rank(t+Rational(1/2)) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_0))]    
    phi_mmu=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        phi_mmu.add_component(x,x,L_P.L_tau(t+Rational(1/2)))
    return phi_mmu    


## finally we need to add the length two maps
## we will write phi_pp_EM... phi_mm_EM for these maps (p=plus, m=minus)
## The ordering of plusses and minuses is (K,mu)
## so phi_pm  is phi^{K,-mu}

def phi_pp_EM(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        if sC_list[x]<t:
            for y in K.v_map.matrix[x]:
                phi.add_component(y,x,L_P.h_sigZ(t-Rational(1/2)))
    return phi
    
def phi_pm_EM(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        if sC_list[x]<t:
            for y in K.v_map.matrix[x]:
                phi.add_component(y,x,L_P.h_tauZ(t-Rational(1/2)))
    return phi
    
def phi_mp_EM(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        if sC_list[x]>t:
            for y in K.h_map.matrix[x]:
                phi.add_component(y,x,L_P.h_sigW(t+Rational(1/2)))
    return phi
    
def phi_mm_EM(K,L_P,s,t):
    sC_list=E_sC_list(K,s,t)
    ranks=[L_P.rank(sC_list[x]) for x in range(len(K.gens_0))]
    co_ranks=[L_P.S_internal.rank for i in range(len(K.gens_1))]
    phi=zero_mor_matrix_ranks(ranks,co_ranks)
    for x in range(len(K.gens_0)):
        if sC_list[x]>t:
            for y in K.h_map.matrix[x]:
                phi.add_component(y,x,L_P.h_tauW(t+Rational(1/2)))
    return phi


### Input:
###   -XK complex K 
###   -KXR L_P
###   -int n
###   -int or Rational N, which is the truncation parameter
###
###output is CFK(P_n(K))
###  


## the following has input a symbol in {"E", "F", "J", "M"}
## as well as a truncation parameter N (int or Rational)
## as well as g\ge 0 (int)
## and s and t (int or Rational)
## the output is the index of Est in the list of all of the cx'es
## ordered lexicographically with lowest value of s and t first
## e.g. E_{s_min, t_min}, E_{s_min,t_min+1} etc
## we order them E, F, J, M

## here is a basic fact which makes the numerology easier. if a<b are
## rational numbers (both ints or both in 1/2+Z) then the number of
## numbers between a and b which differ by 1/2 is equal to b-a

## if a<t and a and t differ by 1/2+Z, then t is the (t-a-1/2)th number
## above a that is greater than a and differs from a by 1/2+Z

## returns -1 if (s,t) are not present in cx

def index_satellite(symbol,n,N,g,s,t):
    if n<0:
        num_Es=4*g*N
        num_Fs=(2*g+1)*2*N
        num_Js=(2*g-n)*(2*N-1)
        if symbol=="E":
            if s<-g or g<s:
                return -1
            if t<-N or N<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=2*g
                s_index_in_row=s+g-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row
        if symbol=="F":
            if s<-g-1 or g<s:
                return -1
            if t<-N or N<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=2*g+1
                s_index_in_row=s+g+1-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es
        if symbol=="J":
            if s<-g+n or g<s:
                return -1
            if t<-N or N-1<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=2*g-n
                s_index_in_row=s+g-n-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es+num_Fs    
        if symbol=="M":
            if s<-g+n-1 or g<s:
                return -1
            if t<-N or N-1<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=2*g-n+1
                s_index_in_row=s+g-n+1-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es+num_Fs+num_Js            
    if n>=0:
        h=max(g,-g+n+1)
        num_Es=(h+g)*2*N
        num_Fs=(h-1+g)*2*N
        num_Js=(h+g-n)*(2*N+1)
        if symbol=="E":
            ## this is the number of s's for a fixed t
            if s<-g or h<s:
                return -1
            if t<-N or N<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=h+g
                s_index_in_row=s+g-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row
        if symbol=="F":
            if s<-g or h-1<s:
                return -1
            if t<-N or N<t:
                return -1
            else:
                num_rows=t+N-Rational(1/2)
                s_per_row=h-1+g
                s_index_in_row=s+g-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es
        if symbol=="J":
            if s<-g+n or h<s:
                return -1
            if t<-N-1 or N<t:
                return -1
            else:
                num_rows=t+N+1-Rational(1/2)
                s_per_row=h+g-n
                s_index_in_row=s+g-n-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es+num_Fs    
        if symbol=="M":
            if s<-g+n or h-1<s:
                return -1
            if t<-N-1 or N<t:
                return -1
            else:
                num_rows=t+N+1-Rational(1/2)
                s_per_row=h-1+g-n
                s_index_in_row=s+g-n-Rational(1/2)
                return num_rows*s_per_row+s_index_in_row+num_Es+num_Fs+num_Js            
    

    ## we now build the satellite complex
    ## N is an element of Rational, as in the paper it is the minimum t so that L_sig\colon C_t\to S is a homotopy equivalence
    
def satellite_truncation(K,L_P,n,N,g):
    ##first we build a list of the E,F,J and M complexes
    ## enumerated so lexicographically
    #E_list is a list of twisted complexes
    E_list=[]
    # E_st is a list of the st values
    E_st_list=[]
    F_list=[]
    F_st_list=[]
    J_list=[]
    J_st_list=[]
    M_list=[]
    M_st_list=[]
    lk=L_P.linking
    if n<0:
        # E_list first
        for y in range(2*N):
            for x in range(2*g):
                s=-g+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)
                E=Est(K,L_P,s,t)
                E.bi_gr_shift([d,d+2*s+2*t-2*(s+t*n)*lk])
                E_list.append(E)
                E_st_list.append([s,t])
        #F_list
        for y in range(2*N):
            for x in range(2*g+1):
                s=-g-1+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)        
                F=Fst(K,L_P,s,t)
                F.bi_gr_shift([d-1,d+2*s-2*(s+t*n)*lk-lk])
                F_list.append(F)
                F_st_list.append([s,t])
        #J_list
        for y in range(2*N-1):
            for x in range(2*g-n):
                s=-g+n+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)        
                J=Jst(K,L_P,s,t)
                J.bi_gr_shift([d-1,d+2*t-2*(s+t*n)*lk])
                J_list.append(J)
                J_st_list.append([s,t])
        #M_list
        for y in range(2*N-1):
            for x in range(2*g-n+1):
                s=-g+n-1+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)        
                M=Mst(K,L_P,s,t)
                M.bi_gr_shift([d-2,d-2-2*(s+t*n)*lk-lk])
                M_list.append(M)
                M_st_list.append([s,t])        
    if n>=0:
        h=max(g,-g+n+1)
        for y in range(2*N):
            for x in range(h+g):
                s=-g+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)
                E=Est(K,L_P,s,t)
                E.bi_gr_shift([d,d+2*s+2*t-2*(s+t*n)*lk])
                E_list.append(E)
                E_st_list.append([s,t])
        #F_list
        for y in range(2*N):
            for x in range(h+g-1):
                s=-g+x+Rational(1/2)
                t=-N+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)
                F=Fst(K,L_P,s,t)
                F.bi_gr_shift([d-1,d+2*s-2*(s+t*n)*lk-lk])
                F_list.append(F)
                F_st_list.append([s,t])
        #J_list
        for y in range(2*N+1):
            for x in range(h+g-n):
                s=-g+n+x+Rational(1/2)
                t=-N-1+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)
                J=Jst(K,L_P,s,t)
                J.bi_gr_shift([d-1,d+2*t-2*(s+t*n)*lk])
                J_list.append(J)
                J_st_list.append([s,t])
        #M_list
        for y in range(2*N+1):
            for x in range(h-1+g-n):
                s=-g+n+x+Rational(1/2)
                t=-N-1+y+Rational(1/2)
                d=Rational(1/4)*(n*(1-(2*t-lk)**2)-2*(1+2*t-lk)*(1+2*s)+4)
                M=Mst(K,L_P,s,t)
                M.bi_gr_shift([d-2,d-2-2*(s+t*n)*lk-lk])
                M_list.append(M)
                M_st_list.append([s,t])
    Cx_list=E_list+F_list+J_list+M_list
    delta_1=mor_matrix([[zero_mor_matrix_Tw(X,Y) for X in Cx_list] for Y in Cx_list])

              
    ## we now build delta_1
    ## we start by adding the \Phi^mu etc. terms to E
    current_index=0
    for x in range(len(E_st_list)):
        st=E_st_list[x]
        s=st[0]
        t=st[1]
        codomain_index=index_satellite("F",n,N,g,s,t)
        if not codomain_index==-1:
            f_mu=phi_mu_EF(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mu)
            
        codomain_index=index_satellite("F",n,N,g,s-1,t)
        if not codomain_index==-1:
            f_mmu=phi_mmu_EF(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mmu)
        
        
        codomain_index=index_satellite("J",n,N,g,s,t)
        if not codomain_index==-1:
            f_K=phi_K_EJ(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_K)        
    
        codomain_index=index_satellite("J",n,N,g,s+n,t-1)
        if not codomain_index==-1:
            f_mK=phi_mK_EJ(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mK) 
        
        codomain_index=index_satellite("M",n,N,g,s,t)
        if not codomain_index==-1:
            f_pp=phi_pp_EM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_pp)        
        
        codomain_index=index_satellite("M",n,N,g,s-1,t)
        if not codomain_index==-1:
            f_pm=phi_pm_EM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_pm)        
        
        codomain_index=index_satellite("M",n,N,g,s+n,t-1)
        if not codomain_index==-1:
            f_mp=phi_mp_EM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mp)
            
        codomain_index=index_satellite("M",n,N,g,s+n-1,t-1)
        if not codomain_index==-1:
            f_mm=phi_mm_EM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mm)
        current_index+=1
        
    # now we add maps to F
    for x in range(len(F_st_list)):
        st=F_st_list[x]
        s=st[0]
        t=st[1]
        
        codomain_index=index_satellite("M",n,N,g,s,t)
        if not codomain_index==-1:
            f_K=phi_K_FM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_K)
        
        codomain_index=index_satellite("M",n,N,g,s+n,t-1)
        if not codomain_index==-1:
            f_mK=phi_mK_FM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mK)        
        current_index+=1
        
    for x in range(len(J_st_list)):
        st=J_st_list[x]
        s=st[0]
        t=st[1]
        
        codomain_index=index_satellite("M",n,N,g,s,t)
        if not codomain_index==-1:
            f_mu=phi_mu_JM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mu)

        codomain_index=index_satellite("M",n,N,g,s-1,t)
        if not codomain_index==-1:
            f_mmu=phi_mmu_JM(K,L_P,s,t)
            delta_1.add_component(codomain_index,current_index,f_mmu)
        
        current_index+=1
    X=Twisted_Complex(Cx_list,delta_1)
    return X.Tot()
        
## same as above except we compute the genus of K, and also use -s1_min from X as N
## here K is an XK object
## X is an KXR object
## n is an integer
## This returns a model of CFK(P(K,n)) from the main paper.
def satellite(K,X,n):
    N=-X.s1_min
    g=max((x[0]-x[1])//2 for x in K.gens_0)
    #(K,L_P,n,N,g)
    return satellite_truncation(K,X,n,N,g)
        

In [7]:
################################
###                          ###
### Examples of  Satellites  ###
###                          ###
################################
# we include some satellite modules of L-spaces


##here we construct the h2_complex of the pq cable of the Hopf link
#we assume the p and q are positive
def h2_cable(p,q):
    numerator=poly(U**(2*p) *V**(2*p*q)-1)
    denominator =poly(U**2 *V**(2*q)-1)
    A=div(numerator,denominator)[0]
    A1=poly(T**0,T)
    A2=Alexander(p,q)
    return Alex_to_h2_poly(p,A,A1,A2)

def KXR_cable(p,q):
    return h2_to_KXR(h2_cable(p,q))




#def XK_cable(K,p,q):
#    if p<= 0 or q<= 0:
#        print("p,q must be positive")
#        return
#    else:
#        return satellite(K, KXR_cable(p,q),0)

##the following function has input an  XK_complex and an output the p,q cable
## output is the CFK complex of K_{p,q}
## The heuristic is actually to write q=pn+r then compute K_{p,q} as
## P_n^{p,r}(K)
## if p<0, then we reverse the signs of both p and q.
## q<0 is okay
## if gcd(p,q) neq 1, then we yell at you


def XK_cable(K,p,q):
    if not gcd(p,q)==1:
        print("gcd(p,q) must be 1")
        return
    else:
        if p<0:
            p=-p
            q=-q
        n=q//p
        r=q%p
    return satellite(K,KXR_cable(p,r),n)


## we additionally have the Whitehead pattern and Mazur patterns
## these are loaded as Mazur(n) and Whitehead

def Mazur_polynomial(n):
    A=poly(-1,U,V)
    for i in range(0,n):
        A=A+poly(U**(2*i) *V**(2*i)*(U**2+V**2))
    for i in range(1,n+1):
        A=A-poly(U**(2*i)* V**(2*i))
    return A


# this returns the mazur satellite of the knot K
## here K is an XK complex
## here m is the type of Mazur pattern
## m\ge 1 is an integer.
## m=1 is the Whitehead pattern
## n is the framing. Can be any integer
def XK_Mazur(K,m,n):
    A=Mazur_polynomial(m)
    A1=poly(T**0,T)
    A2=poly(T**0,T)
    h=Alex_to_h2_poly(m-1,A,A1,A2)
    X=h2_to_KXR(h)
    return satellite(K,X,n)



In [8]:
K=XK_Torus(2,3)

In [9]:
J=CFK_reduced(XK_cable(K,3,2))

In [10]:
J.gens

[[-3, -3],
 [-8, 0],
 [-7, -1],
 [-4, 0],
 [0, -4],
 [-1, -7],
 [0, -8],
 [-3, -1],
 [-2, -2],
 [-2, -2],
 [-1, -3]]

In [11]:
J.diff.matrix

[{3, 4, 8, 9},
 set(),
 {1, 3, 8},
 {7},
 {10},
 {4, 6, 9},
 set(),
 set(),
 {7},
 {10},
 set()]